# Build commit table

The overall goal is to pull commit information about a github project urls listed in URL.txt, then format that info as a table where each row is a separate commit and columns hold info about the commit. Here is the target, where each row represents a single commit:

<img src="https://www.dropbox.com/s/ictvemyemlgux8n/Screenshot%202019-06-28%2010.19.17.png?raw=1">

I first tried to run this from colab. I was able to download the hpcl folder to my desktop from here: https://github.com/HPCL/ideas-uo/tree/master/code/hpcl. Then upload it with code below. I first zipped it on the desktop so you can see unzipping below.

However, I failed to bring in mysql.connector from colab so switched to jupyter server on my desktop

## Assumptions of this notebook

1. The code and notebooks we will be using, including this one, are in the following folder: /ideas-uo/machine_learning/predicting_project_activity

2. We will do a pull from https://github.com/HPCL/ideas-uo to get freshest version of code.

3. Final result will be written to '/ideas-uo/machine_learning/predicting_project_activity' + 'sorted_' + project +  '_commits.csv'.

4. This notebook should be started in folder /ideas-uo/machine_learning/predicting_project_activity.

## Parameters for this notebook

In [1]:
project = 'spack'  #or latte, etc.
project_url = project + '_url.txt'# Where to get the project from github

## Do github pull

Assume that notebook started in context of ideas-uo repository.


In [2]:
#~/ideas-uo/machine_learning/predicting_project_activity
%pwd

'/Users/fickas/Dropbox/boyana/my_work/ideas-uo/machine_learning/predicting_project_activity'

In [3]:
#move up to uo-ideas folder - must be better way
%cd ../..

/Users/fickas/Dropbox/boyana/my_work/ideas-uo


In [4]:
#Refresh local repository
!git pull

Already up-to-date.


In [5]:
import os
repository_dir = os.getcwd() #~/ideas-uo/
code_dir = repository_dir + '/code'
code_dir

'/Users/fickas/Dropbox/boyana/my_work/ideas-uo/code'

In [6]:
import sys
sys.path.append(code_dir)

In [7]:
print( '\n'.join(sys.path))


/Users/fickas/anaconda2/envs/py36/lib/python36.zip
/Users/fickas/anaconda2/envs/py36/lib/python3.6
/Users/fickas/anaconda2/envs/py36/lib/python3.6/lib-dynload
/Users/fickas/anaconda2/envs/py36/lib/python3.6/site-packages
/Users/fickas/anaconda2/envs/py36/lib/python3.6/site-packages/IPython/extensions
/Users/fickas/.ipython
/Users/fickas/Dropbox/boyana/my_work/ideas-uo/code


## Our library

We wrote the hpcl library to act as a github puller. See /ideas-uo/code/hpcl/ for source.

In [8]:
import os, sys, subprocess
from hpcl import Command
from hpcl import GitCommand
#import mysql.connector    #works from command line but not here


In [9]:
#move back to code dir /ideas-uo/code

os.chdir(code_dir)
os.getcwd()

'/Users/fickas/Dropbox/boyana/my_work/ideas-uo/code'

## Note uses "project name"_url.txt to get projects to load

Typically only want to focus on a single project.

Pulling the project info can take some time.

In [10]:
#Thanks to Jason for this code

def checkoutSubrepos(repos,tdir):
    #currdir = os.getcwd()
    for repopath in repos.keys():
        if repopath == tdir: continue
        retcode, out, err = Command.Command('git checkout %s%s' % repos[repopath]).run(dryrun=dry_run)
    return


if __name__ == "__main__":
    
    print('Going to load repos...')

    #Load the list of repos to clone from the following file
    urls = open(os.path.join('.',project_url),'r').readlines()

    #Move to the tmp directory that will hold the repos
    currdir = os.getcwd()
    tmpdir = os.path.join(currdir,'tmp')  #sff: fails if does not already exist
    #os.chdir(tmpdir)

    commander = GitCommand.GitCommand(tmpdir)

    #Download each repo
    for url in urls:
        print('Cloning: ' + url)
        commander.cloneRepo(url)

    print('Finished cloning repos.')

    #Setup DB connection, this will probably change soon.
    #mydb = mysql.connector.connect(host="localhost", port="3307", user="pythondb", passwd="********", database="gitstats")
    #print(mydb)
    #mycursor = mydb.cursor()
    #sql = "INSERT INTO stats (reponame, stats) VALUES(%s, %s)"

    #Now run commands on the repos
    for repo in os.listdir('.'):
        if not os.path.isdir(repo): continue

        print('Checking: ' + repo)
        prefix,versions = commander.getRepoVersions(repo)
        print(prefix)
        print(versions)

        #Gets all the commit data in a dictionary format {authorname:{'total_commits':0, 'commits':{'date':'', 'message':'', 'diffs':{'filename':'', 'diff':''}}}}
        active_developers = commander.getRepoCommitData(repo)

        print(len(active_developers))
        for key,values in active_developers.items():
            print(key)
            #print(values)

        """ stats = getStats('.',repo)
        ts = repo + ', ' + out.strip().split('-')[0] 
        buf = ts
        for i in range(0,len(category_names)): 
            buf += ', %d' % stats[category_names[i]]
            linecounts[i].append(stats[category_names[i]]*0.001)
        outfile.write(buf+'\n')
        #years.append(int(ts))
        print(stats)
        
        val = (repo, buf)"""
        #mycursor.execute(sql,val)
        #mydb.commit()






Going to load repos...
Cloning: https://github.com/spack/spack.git
Finished cloning repos.
Checking: spack

[b'cc76c0f5f9f8021cfb7423a226bd431c00d791ce', b'30d3b32085ab31c91ac45f2f14c5de07774823f7', b'0a0291678e283bf154081df67b0a1f5c909d1d19', b'f5a49280c3b9063c6deb29307cd6356bf75cedd5', b'd32d5e45fb898835343a8aa8853708f60e14bd9f', b'c3b003e6984f15807fe8675d90eab19679566363', b'8540d5390e70388625ee006562b450efa924113b']
git checkout b'cc76c0f5f9f8021cfb7423a226bd431c00d791ce'
b''
git checkout b'30d3b32085ab31c91ac45f2f14c5de07774823f7'
b''
git checkout b'0a0291678e283bf154081df67b0a1f5c909d1d19'
b''
git checkout b'f5a49280c3b9063c6deb29307cd6356bf75cedd5'
b''
git checkout b'd32d5e45fb898835343a8aa8853708f60e14bd9f'
b''
git checkout b'c3b003e6984f15807fe8675d90eab19679566363'
b''
git checkout b'8540d5390e70388625ee006562b450efa924113b'
b''
561
b'Phil Tooley <32297355+ptooley@users.noreply.github.com>'
b'Mark Olesen <Mark.Olesen@esi-group.com>'
b'Phil Carns <carns@mcs.anl.gov>'
b'Toyohis

## Let's see what we have

In [11]:
len(active_developers)  #465 separate people doing commits on the project

561

In [12]:
dev_tups = list(active_developers.items())  #get it in form easier to see

In [13]:
type(dev_tups)

list

In [14]:
x = dev_tups[0]
x

(b'Phil Tooley <32297355+ptooley@users.noreply.github.com>',
 {'commits': [{'date': b'Thu Jul 11 21:55:43 2019 +0100',
    'diffs': ['+++ b/var/spack/repos/builtin/packages/dimemas/package.py',
     '+# Copyright 2013-2019 Lawrence Livermore National Security, LLC and other',
     '+# Spack Project Developers. See the top-level COPYRIGHT file for details.',
     '+#',
     '+# SPDX-License-Identifier: (Apache-2.0 OR MIT)'],
    'message': '    New package: dimemas (#11979)\n'},
   {'date': b'Wed Feb 27 00:41:44 2019 +0000',
    'diffs': ['+++ b/var/spack/repos/builtin/packages/routino/package.py',
     '+# Copyright 2013-2019 Lawrence Livermore National Security, LLC and other',
     '+# Spack Project Developers. See the top-level COPYRIGHT file for details.',
     '+#',
     '+# SPDX-License-Identifier: (Apache-2.0 OR MIT)'],
    'message': '    new package: routino (#10679)\n'},
   {'date': b'Tue Nov 13 06:14:29 2018 +0000',
    'diffs': ['+++ b/var/spack/repos/builtin/packages/openi

In [15]:
x[0]

b'Phil Tooley <32297355+ptooley@users.noreply.github.com>'

In [16]:
x[1].keys()

dict_keys(['total_commits', 'commits'])

In [17]:
x[1]['total_commits']

35

In [18]:
z = x[1]['commits']
len(z)

35

In [19]:
z[0]

{'date': b'Thu Jul 11 21:55:43 2019 +0100',
 'diffs': ['+++ b/var/spack/repos/builtin/packages/dimemas/package.py',
  '+# Copyright 2013-2019 Lawrence Livermore National Security, LLC and other',
  '+# Spack Project Developers. See the top-level COPYRIGHT file for details.',
  '+#',
  '+# SPDX-License-Identifier: (Apache-2.0 OR MIT)'],
 'message': '    New package: dimemas (#11979)\n'}

## Example of 5 commits

<pre>
[{'date': b'Fri Jul 5 19:57:00 2019 -0700',
  'diffs': ['+import traceback'],
  'message': "    bugfix: always generate a stack trace when spack is run with --debug (#11940)\n    \n    - We weren't previously printing stack traces on SystemExit or\n      KeyboardInterrupts.\n    \n    - Either raise or print the stacktrace in these cases.\n"},
 {'date': b'Wed Jul 3 01:56:13 2019 -0700',
  'diffs': ['+  - share/spack/qa/.*',
   '+  - share/spack/spack-completion.bash'],
  'message': '    tests: add tests for setup-env.sh\n    \n    - tests use a shell-script harness and test all Spack commands that\n      require special shell support.\n    \n    - tests work in bash, zsh, and dash\n    \n    - run setup-env.sh tests on macos and linux builds.\n      - we run them on macos and linux\n'},
 {'date': b'Wed Jul 3 17:11:07 2019 -0700',
  'diffs': ['-      default: true',
   '-      llnl:',
   '+      default:',
   '-        paths:',
   '-          - lib/spack/llnl',
   '-      commands:',
   '-        threshold: 0.5',
   '-        paths:',
   '-          - lib/spack/spack/cmd',
   '-      build_systems:',
   '-        threshold: 0.5',
   '-        paths:',
   '-          - lib/spack/spack/build_systems',
   '-      modules:',
   '-        threshold: 0.5',
   '-        paths:',
   '-          - lib/spack/spack/modules',
   '-      core:',
   '-        threshold: 0.5',
   '-        paths:',
   '-          - "!lib/spack/llnl"',
   '-          - "!lib/spack/spack/cmd"'],
  'message': "    coverage: remove sub-reports; only show project/patch on PRs (#11926)\n    \n    - There's too much information on our PRs, and the Travis test results\n      (which are arguably more important than coverage) get buried in\n      coverage information\n    \n    - Remove coverage sub-categories, as you can browse codecov's file view\n      to see roughly the same thing\n    \n    - show only project/patch on GitHub PRs.\n"},
 {'date': b'Sun Jun 16 17:29:25 2019 +0200',
  'diffs': ['+++ b/var/spack/repos/builtin/packages/cowsay/package.py',
   '+# Copyright 2013-2019 Lawrence Livermore National Security, LLC and other',
   '+# Spack Project Developers. See the top-level COPYRIGHT file for details.',
   '+#',
   '+# SPDX-License-Identifier: (Apache-2.0 OR MIT)'],
  'message': '    cowsay: new package (#11743)\n    \n    - A program that generates ASCII pictures of a cow with a message.\n    \n    - This is used by the ISC19 Singularity tutorial, so we should probably\n      have it in Spack.\n'},
 {'date': b'Thu Jun 6 14:07:46 2019 -0700',
  'diffs': ['-            self.stage.path,',
   "-            'luafilesystem-{version.underscored}',",
   '+            self.stage.source_path,'],
  'message': '    bugfix: lua-luafilesystem package should use stage.source_path (#11648)\n'}]
</pre>

In [20]:
import re

In [21]:
def count_changes(change_list):
    change_count = 0
    #Go through each diff represented as a string, e.g., '+  - share/spack/qa/.*'
    for change in change_list:
        #First get the leading part of the string up to first non-change.
        #Note: gave up on re.search as too complicated.
        alts = '+- '  #3 ok chars
        front = ''
        for c in change:
            if c not in alts: break
            if not c.isspace(): front += c  #don't include spaces
        wrangled_change = re.sub(r"\+\s*\-", "|", front)  #capture replaces as a + - pair
        change_count += len([1 for char in wrangled_change if char in '+-|'])  #use re.findall instead?
    return change_count
        

I am going to reformat the data. Currently the key is user and the value is a nested structure that includes all the commits for that person. I want to invert this. I'd like a list of commits, where each commit has info including person.

Note Python is a mess when it comes to time zones. What is supposed to work, %z, does not. See discussion here: https://stackoverflow.com/questions/3305413/python-strptime-and-timezones.

In particular, these fail when include UTC: '%a %b %m %X %y %z' and '%c %z'.

I am resorting to a utility package.

In [22]:
import datetime
from dateutil import parser

In [23]:
x = parser.parse("Tue Aug 2 14:58:31 2016 +0200")  #+0200 hours => 7200 seconds
x  #just an example

datetime.datetime(2016, 8, 2, 14, 58, 31, tzinfo=tzoffset(None, 7200))

In [24]:
x.utcoffset().seconds

7200

In [25]:
all_commits = []

for entry in dev_tups:
    name = entry[0]                               #'Gilles Fourestey <gilles.fourestey@epfl.ch>'
    commit_list = entry[1]['commits']      #a list of dictionaries with date, diffs, message as keys
    seen = []                               #need this because getting duplicates
    for a_commit in commit_list:
        if a_commit in seen: continue
        if not a_commit['diffs']: continue  #skip commit with empty diff
        seen.append(a_commit)
        new_dict = {}
        date = parser.parse(a_commit['date'])         #"Tue Aug 2 14:58:31 2016 +0200"
        new_dict['year'] = date.year
        new_dict['doy'] = min(365, int(date.strftime('%j'))) #punting on leap year
        new_dict['month'] = date.month
        new_dict['day_of_month'] = date.day
        new_dict['day_name'] = date.strftime('%A')  #e.g., Tuesday
        new_dict['utc_offset'] = date.utcoffset().seconds  #e.g., 7200

        diffs = a_commit['diffs']  #list of strings
        new_dict['locc'] = total_changes = count_changes(diffs)

        new_dict['name'] = name
        new_dict['message'] = a_commit['message']             #'Fixed broken link in README (#1733)\n'
        all_commits.append(new_dict)
        
    

In [26]:
len(all_commits) #10913 commits spread over roughly 6 years

4972

In [27]:
all_commits[:5]

[{'day_name': 'Thursday',
  'day_of_month': 11,
  'doy': 192,
  'locc': 7,
  'message': '    New package: dimemas (#11979)\n',
  'month': 7,
  'name': b'Phil Tooley <32297355+ptooley@users.noreply.github.com>',
  'utc_offset': 3600,
  'year': 2019},
 {'day_name': 'Wednesday',
  'day_of_month': 27,
  'doy': 58,
  'locc': 7,
  'message': '    new package: routino (#10679)\n',
  'month': 2,
  'name': b'Phil Tooley <32297355+ptooley@users.noreply.github.com>',
  'utc_offset': 0,
  'year': 2019},
 {'day_name': 'Tuesday',
  'day_of_month': 13,
  'doy': 317,
  'locc': 7,
  'message': '    New package: openimageio (#9715)\n',
  'month': 11,
  'name': b'Phil Tooley <32297355+ptooley@users.noreply.github.com>',
  'utc_offset': 0,
  'year': 2018},
 {'day_name': 'Tuesday',
  'day_of_month': 13,
  'doy': 317,
  'locc': 7,
  'message': '    New package: dcmtk (#9716)\n',
  'month': 11,
  'name': b'Phil Tooley <32297355+ptooley@users.noreply.github.com>',
  'utc_offset': 0,
  'year': 2018},
 {'day_na

Looks ok to me. Now converting to pandas dataframe is easy peasy.

In [28]:
import pandas as pd

In [29]:
df_commits = pd.DataFrame(all_commits)  #gotta love pandas

In [30]:
df_commits.head()

,day_name,day_of_month,doy,locc,message,month,name,utc_offset,year
0,Thursday,11,192,7,New package: dimemas (#11979)\n,7,b'Phil Tooley <32297355+ptooley@users.noreply....,3600,2019
1,Wednesday,27,58,7,new package: routino (#10679)\n,2,b'Phil Tooley <32297355+ptooley@users.noreply....,0,2019
2,Tuesday,13,317,7,New package: openimageio (#9715)\n,11,b'Phil Tooley <32297355+ptooley@users.noreply....,0,2018
3,Tuesday,13,317,7,New package: dcmtk (#9716)\n,11,b'Phil Tooley <32297355+ptooley@users.noreply....,0,2018
4,Tuesday,2,183,2,openfoam: rename openfoam-com to openfoam ...,7,b'Mark Olesen <Mark.Olesen@esi-group.com>',3600,2019


In [31]:
#df_sorted = df_commits.sort_values(by='date')  #fails because of UTC
df_sorted = df_commits.sort_values(['year', 'doy'])  #primary and secondary
df_sorted = df_sorted.reset_index(drop=True)

In [32]:
df_sorted.head(50)

,day_name,day_of_month,doy,locc,message,month,name,utc_offset,year
0,Wednesday,13,44,4,Initial version of spack with one package:...,2,b'Todd Gamblin <tgamblin@llnl.gov>',57600,2013
1,Monday,18,49,2,Require python2.7\n,2,b'Todd Gamblin <tgamblin@llnl.gov>',57600,2013
2,Monday,18,49,2,"Dependencies now work. Added libelf, libd...",2,b'Todd Gamblin <tgamblin@llnl.gov>',57600,2013
3,Wednesday,20,51,1,Added libunwind and fixed link issues in c...,2,b'Todd Gamblin <tgamblin@llnl.gov>',57600,2013
4,Thursday,21,52,1,Minor changes; loosened up parallel build ...,2,b'Todd Gamblin <tgamblin@llnl.gov>',57600,2013
5,Friday,22,53,9,Better spack -h: added cmd descriptions.\n...,2,b'Todd Gamblin <tgamblin@llnl.gov>',57600,2013
6,Monday,25,56,8,Simpler implementation of depends_on.\n,2,b'Todd Gamblin <tgamblin@llnl.gov>',57600,2013
7,Monday,25,84,12,Moved install-spack to its own simpler com...,3,b'Todd Gamblin <tgamblin@llnl.gov>',61200,2013
8,Monday,25,84,12,Adding install script\n,3,b'Todd Gamblin <tgamblin@llnl.gov>',61200,2013
9,Thursday,9,129,11,Removed old versions.py\n,5,b'Todd Gamblin <tgamblin@llnl.gov>',61200,2013


In [33]:
df_sorted.tail()

,day_name,day_of_month,doy,locc,message,month,name,utc_offset,year
4967,Monday,8,189,7,New package: r-grbase (starting at 1.8-3.4...,7,b'Justin S <3630356+codeandkey@users.noreply.g...,68400,2019
4968,Tuesday,9,190,8,package: Add powerapi package (#11845)\n ...,7,b'Carson Woods <carsonwoods@users.noreply.gith...,64800,2019
4969,Tuesday,9,190,2,motif package: fix the download url (#1193...,7,b'Hironori-Yamaji <52182908+Hironori-Yamaji@us...,32400,2019
4970,Thursday,11,192,7,New package: dimemas (#11979)\n,7,b'Phil Tooley <32297355+ptooley@users.noreply....,3600,2019
4971,Thursday,11,192,3,libfabric package: add support for tcp pro...,7,b'Phil Carns <carns@mcs.anl.gov>',72000,2019


In [34]:
len(df_sorted)

4972

In [35]:
foo_fie_fum()  #break here from Run All and decide if want to write out

NameError: name 'foo_fie_fum' is not defined

Write it out to file

In [36]:
save_dir = repository_dir + '/machine_learning/predicting_project_activity/'

In [37]:
df_sorted.to_csv(save_dir + 'sorted_' + project +  '_commits.csv', index=False)